# Notebook Information

This notebook demonstrates how to implement simgle RAG over structured data - SQL database.


---

This notebook is maintained by:

**Name:** Adam Krzysiek 
**Email:** [adam_krzysiek@epam.com](adam_krzysiek@epam.com)

---

In [ ]:
%pip install --upgrade pip > /dev/null
%pip install langchain-core langchain-experimental langchain-openai > /dev/null

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate


## Load env

In [9]:
load_dotenv(find_dotenv())

if not os.environ.get("OPENAI_API_KEY"):
    raise Exception("Missing OPENAI_API_KEY")


## Setup llm

In [10]:
llm = AzureChatOpenAI(
  api_key         = os.environ['OPENAI_API_KEY'],
  api_version     = "2023-07-01-preview",
  azure_endpoint  = "https://ai-proxy.lab.epam.com",
  model           = "gpt-4o-mini-2024-07-18",
  temperature     = 0.0
)

### Test LLM

In [12]:
llm.invoke("What is the capital of Poland?").content

KeyboardInterrupt: 

## Database setup

In [ ]:
db = SQLDatabase.from_uri("sqlite:///data/cars.db")

### Test database

In [ ]:
print(f"Database dialect: {db.dialect}")
print(f"Available table names: {db.get_usable_table_names()}")
db.run("SELECT * FROM cars LIMIT 10;")

## Database chain

The SQLDatabaseChain class is designed for interacting with SQL databases. It allows natural language queries to be translated 
into SQL queries and executed against a database to get result.

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True, return_sql=False, return_direct=True)


### Test database chain

Let's try some queries.

In [ ]:
db_chain("get cars")['result']

In [ ]:
db_chain("give me 3 most expensive cars")

In [ ]:
db_chain.run("give me cars without AC")

In [ ]:
db_chain("get cars worth more than 1 milion")['result']

## RAG

In [ ]:
RAG_PROMPT_TEMPLATE = """
You are professional AI car recommendation assistant. Given the human query as HUMAN_QUERY, your goal is to provide car recommendation matching
expectations enclosed in there, based on cars available listed under AVAILABLE_CARS. If AVAILABLE_CARS is emptly it simply means there are not cars to be recommended.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            RAG_PROMPT_TEMPLATE,
        ),
        (
            "human",
            "HUMAN_QUERY:{human_query}\nAVAILABLE_CARS:{available_cars}"
        ),
    ]
)

chain = prompt | llm

### Example query

In [ ]:
query = "I am looking for SUV and cheap cars with AC (order by price)"
available_cars = db_chain.run(query)
resp = chain.invoke(
    {
        "available_cars": available_cars,
        "human_query": query,
    }
)

print(resp.content)